In [1]:
import pandas as pd
import numpy as np
from six.moves import xrange
import math
import seaborn as sns
import matplotlib.pyplot as plt
import random
import pickle

### load data

In [2]:
all_data = pd.read_csv('../data/step1_clear_data.csv')
all_data.head()

,day,exc_num,exc_times,uid,velocity,force,front,torque_xy,tt_freq,Lx,Ly,Az,avg_velocity
0,1,1.1,1,4,0.021297,18.561272,0.000039,-0.192111,0.0,0.021297,0.0,0.006509,0.021297
1,1,1.1,1,4,0.021297,18.561272,0.000078,-0.192111,0.0,0.021297,0.0,0.006509,0.021297
2,1,1.1,1,4,0.023768,23.049513,0.000120,-1.395340,1.0,0.023768,0.0,0.005268,0.022121
3,1,1.1,1,4,0.023768,23.049513,0.000164,-1.395340,1.0,0.023768,0.0,0.005268,0.022532
4,1,1.1,1,4,0.031793,32.181287,0.000206,-2.664103,1.0,0.031793,0.0,0.000000,0.024384


In [33]:
def ability_level_mapper(data, groups=None, col='front', how='naive', n_level=19, invert=True, parameters=None):
    # the raw data is divided into groups according to its exc_num, ability levels are calculated respectively
    # how: 1 is mapping without any other processing, called 'naive'
    origin = data.copy()
    
    if parameters is not None:
        col, how, n_level, invert, v_max, v_min = parameters
        interval = (v_max- v_min)/n_level
        assert interval!=0, 'zero dividend'
        origin['performance'] = (origin[col]-v_min)/interval
        
        origin['performance'] = origin['performance'].astype(int)
        if invert:
            origin['performance'] = n_level-origin['performance']+1
        else:
            origin['performance'] = origin['performance']+1
        
        return origin, parameters
        
    if groups == None:
        v_max = origin[col].max()
        v_min = origin[col].min()
        
        interval = (v_max- v_min)/n_level
        assert interval!=0, 'zero dividend'
        origin['performance'] = (origin[col]-v_min)/interval
        
        origin['performance'] = origin['performance'].astype(int)
        if invert:
            origin['performance'] = n_level-origin['performance']+1
        else:
            origin['performance'] = origin['performance']+1
        
        parameters = (col, how, n_level, invert, v_max, v_min )
        return origin, parameters
    
    if how == 1 or how=='naive':
        tmp = origin[col]
        for index, group in groups:
            
            v_max = group[col].max()
            v_min = group[col].min()
            
            interval = (v_max-v_min)/n_level
            
            assert interval!=0, 'zero dividend'
            
            origin.loc[index, col] = (origin.loc[index, col]-v_min)/interval
        origin[col] = origin[col].astype(int)
        if invert:
            origin['performance'] = n_level-origin[col]+1
            
        else:
            origin['performance'] = origin['performance']+1
        origin[col] = tmp
        parameters = (col, how, n_level, invert, v_max, v_min )
        return origin, parameters
    
def cMGRM(theta, alpha=1.5, b=0.5, c=0):
    # MGRM cumulative probability
    return math.e**(alpha*(theta-b+c))/(1+math.e**(alpha*(theta-b+c)))


### select data

In [13]:
# uid, day, exc, times
selected_uid_day_exc_times = (7, 2, 0.1, 1)
selected_data = all_data.set_index(['uid', 'day', 'exc_num', 'exc_times']).loc[selected_uid_day_exc_times]
len(selected_data)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  This is separate from the ipykernel package so we can avoid doing imports until


1154

In [23]:
selected_data = selected_data.reset_index()
selected_data.head()

,uid,day,exc_num,exc_times,velocity,force,front,torque_xy,tt_freq,Lx,Ly,Az,avg_velocity
0,7,2,0.1,1,0.260915,34.591517,0.000074,7.293841,1.0,0.259794,-0.024163,0.031698,0.328289
1,7,2,0.1,1,0.260915,34.591517,0.000099,7.293841,1.0,0.259794,-0.024163,0.031698,0.318473
2,7,2,0.1,1,0.269090,35.357908,0.000123,6.242783,1.0,0.267447,-0.029696,0.041651,0.309065
3,7,2,0.1,1,0.269090,35.357908,0.000147,6.242783,1.0,0.267447,-0.029696,0.041651,0.299658
4,7,2,0.1,1,0.269090,35.357908,0.000172,6.242783,1.0,0.267447,-0.029696,0.041651,0.293520


### parameters 

In [22]:
alpha = 1.606998
b = -0.755847
cl = [1.359, 1.1, 0.919, 0.782, 0.62, 0.418, 0.388, 0.075, -0.054, -0.056, 
          -0.059, -0.103, -0.156, -0.394, -0.415,
          -0.478, -1.377, -1.471, -1.989]

levels = [i for i in xrange(18)]
abilities = [-4]
for level in levels:
    curr = b-(cl[level]+cl[level+1])
    abilities.append(curr)
abilities.append(4)
abilities

[-4,
 -3.2148470000000002,
 -2.7748470000000003,
 -2.4568470000000002,
 -2.1578470000000003,
 -1.793847,
 -1.5618470000000002,
 -1.218847,
 -0.7768470000000001,
 -0.6458470000000001,
 -0.640847,
 -0.593847,
 -0.49684700000000004,
 -0.205847,
 0.053152999999999895,
 0.13715299999999997,
 1.0991529999999998,
 2.0921529999999997,
 2.704153,
 4]

### online evaluation

In [52]:
all_abilities = []
with open('../data/parameter/ability_mapper.p', 'rb') as f:
    parameters = pickle.load(f)
for i in xrange(len(selected_data)):
    tmp = selected_data.loc[i]
    tmp,_ = ability_level_mapper(tmp, parameters=parameters)
    perf = tmp['performance']
    print('input is:' + str(perf))
    theta = abilities[int(perf)]
    print('ability degree is:' + str(theta))
    all_abilities.append(theta)

input is:5.0
ability degree is:-1.793847
input is:5.0
ability degree is:-1.793847
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003


input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:-2.1578470000000003
input is:4.0
ability degree is:

input is:2.0
ability degree is:-2.7748470000000003
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:3.0
ability degree is:-2.4568470000000002
input is:4.0
ability degree is:

input is:5.0
ability degree is:-1.793847
input is:5.0
ability degree is:-1.793847
input is:5.0
ability degree is:-1.793847
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6.0
ability degree is:-1.5618470000000002
input is:6

input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:-3.2148470000000002
input is:1.0
ability degree is:

In [53]:
np.mean(all_abilities)

-2.0338114714038129